# Finetuning the dataset using gpt-3.5-turbo-0125 for 10 epochs

In [1]:
!pip install openai==1.12.0
!pip install pandas
!pip install beautifulsoup4
!mkdir train_data
!pip install python-dotenv



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
mkdir: train_data: File exists

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
import json


from openai import OpenAI
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()
# Access the API key from the environment
openai_api_key = os.getenv("OPENAI_API_KEY")

train_data=pd.read_csv("Finetune_data_125.csv",encoding='latin1')

final_data={}
for each_data in train_data.iterrows():
    final_data[each_data[1][8]]={"Indicators":each_data[1][0],
            #   "Description":each_data[1][1],
              "Geographic_Coverage":each_data[1][2],
              "Format":each_data[1][3],
              "Spatial_Resolution":each_data[1][4],
              "Temporal_Resolution":each_data[1][5],
              "Temporal_Extent":each_data[1][6],
              "Latency":each_data[1][7],
                "Project":each_data[1][9],
                "Data_Visualization":each_data[1][10]}
    
for url,value in final_data.items():
    response=requests.get(url)
    html_page = response.text
    soup = BeautifulSoup(html_page, "html.parser")
    text=soup.get_text()
    text = re.sub(r'[\t\n\r\f\v]+', '', text)
    text=re.sub(r'[^\w\s]', '', text)
    final_data[url]["text"]=text

_SYSTEM_PROMPT ="Extract metadata and entities details accurately from my requests." + "Metadata extraction for Indicators should clearly be one of the element in the list ['Disasters','Human Dimensions','Food Availability','Health & Air Quality','Water Availability', 'Extreme Heat','Urban Flooding','Climate']. Do not create new element for indicators " + "Make sure the following fields follow the following regex pattern"+"Spatial_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$"+"Temporal_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$"+"Temporal_Extent: ^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$"


train_data=[]
for url,value in final_data.items():
    # output_content={"Indicators":value["Indicators"],"Description":value["Description"],"Geographic_Coverage":value["Geographic_Coverage"], "Format":value["Format"],"Spatial_Resolution":value["Spatial_Resolution"],"Temporal_Resolution":value["Temporal_Resolution"],"Temporal_Extent":value["Temporal_Extent"],"Latency":value["Latency"],"Project":value["Project"],"Data_Visualization":value["Data_Visualization"]}
    output_content={"Indicators":value["Indicators"],"Geographic_Coverage":value["Geographic_Coverage"], "Format":value["Format"],"Spatial_Resolution":value["Spatial_Resolution"],"Temporal_Resolution":value["Temporal_Resolution"],"Temporal_Extent":value["Temporal_Extent"],"Latency":value["Latency"],"Project":value["Project"],"Data_Visualization":value["Data_Visualization"]}
    output_content=str(output_content)
    data={"messages": [{"role": "system", "content":_SYSTEM_PROMPT},{"role": "user", "content":f'{value["text"]}'}, {"role": "assistant", "content": output_content}]}
    train_data.append(data)
    
with open("train_data_125_without_description.jsonl", 'w') as jsonl_file:
    for record in train_data:
        jsonl_file.write(json.dumps(record) + '\n')

import pickle
train_125_urls=list(final_data.keys())
with open("train_data/train_125_urls_without_description.pkl",'wb') as file:
    pickle.dump(train_125_urls,file)


client = OpenAI(api_key=openai_api_key)

client.files.create(
  file=open("train_data_125_without_description.jsonl", "rb"),
  purpose="fine-tune"
)

/var/folders/m4/9_ydsvdn3b3d84rntrrsxm0h0000gn/T/ipykernel_29242/3261597445.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  final_data[each_data[1][8]]={"Indicators":each_data[1][0],
/var/folders/m4/9_ydsvdn3b3d84rntrrsxm0h0000gn/T/ipykernel_29242/3261597445.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "Geographic_Coverage":each_data[1][2],
/var/folders/m4/9_ydsvdn3b3d84rntrrsxm0h0000gn/T/ipykernel_29242/3261597445.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

FileObject(id='file-FLMunoYmrkXBfD4AyoFxbfDo', bytes=775517, created_at=1709045965, filename='train_data_125_without_description.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [3]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
import json
client.fine_tuning.jobs.create(
  training_file="file-FLMunoYmrkXBfD4AyoFxbfDo", 
  model="gpt-3.5-turbo-1106", 
  hyperparameters={
    "n_epochs":10
  }
)

FineTuningJob(id='ftjob-bDh8kRLqdPvH474MGFZvQecK', created_at=1709046159, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=10, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-KVpMEM9Q8Xm2pc8YuAMYw8r3', result_files=[], status='validating_files', trained_tokens=None, training_file='file-FLMunoYmrkXBfD4AyoFxbfDo', validation_file=None, user_provided_suffix=None)

In [8]:
# List the top 1 fine-tuning jobs
client.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-bDh8kRLqdPvH474MGFZvQecK', created_at=1709046159, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-1106:nasa::8wu2oCdL', finished_at=1709049150, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-KVpMEM9Q8Xm2pc8YuAMYw8r3', result_files=['file-T8kiLwTkA2Kgu1ZbfRI3fpbx'], status='succeeded', trained_tokens=1676880, training_file='file-FLMunoYmrkXBfD4AyoFxbfDo', validation_file=None, user_provided_suffix=None)], object='list', has_more=True)